In [69]:
import numpy as np
import random #https://stackoverflow.com/questions/306400/how-to-randomly-select-an-item-from-a-list
import math

In [70]:
source = [8,4,12,6]
dest = [3,5,10,7,5]
supply_coords = [[7., 6.],[2., 7.],[1., 6.],[6., 8.]]
demand_coords = [[3., 9.],[4., 6.],[5., 7.],[5., 3.],[2., 9.]]
 
def distance(x1,x2,y1,y2):
    return math.sqrt((x1-x2)**2+(y1-y2)**2)

#cost function
cost = np.zeros((len(source),len(dest)))
for i in range(len(source)):
    for j in range(len(dest)):
        cost[i,j] = distance(supply_coords[i][0],demand_coords[j][0],supply_coords[i][1],demand_coords[j][1])

#population number controls how many parents are produced. For now lets keep as 2.        
population = 10

def bubble_sort(arrx):   
    swapped = True
    while swapped:
        swapped = False
        for i in range(len(arrx) - 1):
            if arrx[i][0] > arrx[i + 1][0]:
                arrx[i], arrx[i + 1] = arrx[i + 1], arrx[i]
                swapped = True
                
#Initialisation
pi = []
for i in range(1,len(source)*len(dest)+1):
    pi.append(i)


parents= []

for p in range(population):

    X = np.zeros((len(source),len(dest)))

    s = source.copy()
    d = dest.copy()

    test = pi.copy()

    while(len(test)!=0):
        k = random.choice(test)
        i = int(((k-1)/len(d)))
        j = ((k-1)%len(d)) 
        X[i,j] = min(s[i],d[j])
        s[i] = s[i] - X[i,j]
        d[j] = d[j] - X[i,j]
        test.remove(k)           #https://www.programiz.com/python-programming/methods/list/remove
        
    parents.append([np.sum(cost*X),X])
    
#     if(np.sum(cost*X) < minval):
#         minval = np.sum(cost*X)
#         answer = np.array(X)

#sorts the parent based on their costs
bubble_sort(parents)
print(parents[0][0])
#parents = parents[:len(parents)//2]

93.9537889375


In [71]:
def offsprings_maker(p,q):

    D = np.zeros((len(source),len(dest)))
    R = np.zeros((len(source),len(dest)))

    for i in range(len(source)):
        for j in range(len(dest)):
            D[i,j] = int((p[i,j]+q[i,j])/2)
            R[i,j] = (p[i,j]+q[i,j])%2

    R1 = np.zeros((len(source),len(dest)))
    R2 = np.zeros((len(source),len(dest)))

    flag = 0
    for i in range(len(source)):
        for j in range(len(dest)):
            if(R[i,j]==1):
                if(flag==0):
                    R1[i,j]= 1
                    flag = 1
                else:
                    R2[i,j]= 1
                    flag = 0
    X1 = D+R1
    X2 = D+R2
    
    return X1,X2


In [72]:
offsprings = []
copy_parents = parents.copy()
while len(copy_parents) > 1 :
    p = copy_parents[0]
    q = copy_parents[-1]
    off1 =[]
    off2 = []
    a = p[1]
    b = q[1]
    off1,off2 = offsprings_maker(a,b)
    offsprings.append(off1)
    offsprings.append(off2)
    copy_parents.remove(p)
    copy_parents.remove(q) 

In [73]:
def mutate(a):
    n_rows, n_cols = random.randint(2,a.shape[0]),random.randint(2,a.shape[1])
#    n_rows, n_cols = random.randint(2,a.shape[0])//random.randint(1,a.shape[0]//2),random.randint(2,a.shape[1])//random.randint(1,a.shape[1]//2)
    row = []
    col = []
#     if n_rows==0:
#         n_rows = 1
#     if n_cols==0:
#         n_cols = 1
    while len(row)<n_rows:
        x = random.randint(0,a.shape[0]-1)
        if x not in row:
            row.append(x)
    while len(col)<n_cols:
        x = random.randint(0,a.shape[1]-1)
        if x not in col:
            col.append(x)
    row.sort()
    col.sort()
    
    A = np.zeros((n_rows,n_cols))
    
    s = np.sum(a[np.ix_(row,col)],axis=1)
    d = np.sum(a[np.ix_(row,col)],axis=0)
    
    test = []
    for i in range(1,len(s)*len(d)+1):
        test.append(i)
    
    while(len(test)!=0):
        k = random.choice(test)
        i = int(((k-1)/len(d)))
        j = ((k-1)%len(d)) 
        A[i,j] = min(s[i],d[j])
        s[i] = s[i] - A[i,j]
        d[j] = d[j] - A[i,j]
        test.remove(k)           #https://www.programiz.com/python-programming/methods/list/remove
    
    row_itr = 0
    col_itr = 0
    for i in row:
        for j in col:
            a[i,j] = A[row_itr,col_itr]            
            col_itr = col_itr+1
        row_itr = row_itr+1
        col_itr=0
    #Mutated offspring
    return a

In [74]:
minval = 3000
for i in offsprings:
    if minval>np.sum(cost*i):
        minval = np.sum(cost*i)
mv = 3000
for i in offsprings:
    i= mutate(i)
    if mv>np.sum(cost*i):
        mv = np.sum(cost*i)
print(minval)
print(mv)


112.142639641
89.404829273
